In [1]:
# %%
from torch.profiler import profile, record_function, ProfilerActivity
import torch
import total_energy as te
from mala.network import TesterGraph
from mala import printout
import mala
import os


parameters = mala.Parameters()


parameters.data.data_splitting_type = "by_snapshot"
parameters.data.use_graph_data_set = True

# Specify the training parameters.
parameters.running.trainingtype = "Adam"
parameters.running.visualisation = 1
parameters.running.visualisation_dir = "gnn_tests_tensorboard"


parameters.targets.ldos_gridsize = 201
parameters.targets.ldos_gridoffset_ev = -13.5
parameters.targets.ldos_gridspacing_ev = 0.1
parameters.targets.pseudopotential_path = "/bigdata/casus/wdm/Bartek_H2/H128"

parameters.verbosity = 2

parameters.use_gpu = True

# ! HYPERPARAMETERS
parameters.running.max_number_epochs = 20
parameters.running.ldos_grid_batch_size = 30
parameters.running.learning_rate = 0.00001

parameters.data.n_closest_ldos = 32
parameters.data.n_closest_ions = 8

# ! HYPERPARAMETERS

In [2]:
model_path = "H2_model_big_01"
parameters, network, data_handler, tester = TesterGraph.load_run(model_path)

for i in range(0, 1): # Testing on train snapshots
    data_handler.add_snapshot(
        f'H_snapshot{i}.pw.scf.in', f'/bigdata/casus/wdm/Bartek_H2/H128/snapshot{i}',
        f'H_snapshot{i}.out.npy', '/bigdata/casus/wdm/Bartek_H2/H128/ldos/',
        'te', calculation_output_file=f'/bigdata/casus/wdm/Bartek_H2/H128/outputs/snapshot{i}.out'
    )

data_handler.prepare_data(reparametrize_scaler=False)

Checking the snapshots and your inputs for consistency.
DataHandler prepared for inference. No training possible with this setup. If this is not what you wanted, please revise the input script. Validation snapshots you may have entered willbe ignored.
Consistency check successful.
Build datasets.
Using GraphDataset.


  0%|          | 0/1 [00:00<?, ?it/s]

Build dataset: Done.


In [3]:
network = network.to('cuda')
observables_to_test = [
    "ldos",
    "band_energy",
    "band_energy_full",
    "number_of_electrons",
    "total_energy",
    "total_energy_full",
    "density",
    "dos"
]
tester.observables_to_test = observables_to_test
results = tester.test_all_snapshots()
print(results)

  0%|          | 0/10000 [00:00<?, ?it/s]

/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/home/brzoza73/casus/mala-venv/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


{'ldos': [0.008783927603911687], 'band_energy': [inf], 'band_energy_full': [inf], 'number_of_electrons': [-442.78248985367185], 'total_energy': [inf], 'total_energy_full': [inf], 'density': [inf], 'dos': [nan]}


/home/brzoza73/casus/mala/mala/network/tester_base.py:297: RuntimeWarning: divide by zero encountered in divide
  return np.mean(np.abs((actual - predicted) / actual)) * 100
/home/brzoza73/casus/mala/mala/network/tester_base.py:297: RuntimeWarning: invalid value encountered in divide
  return np.mean(np.abs((actual - predicted) / actual)) * 100
